In [1]:
import os
from pathlib import Path
import json
import numpy as np
import torch
import torch.nn as nn
import gym
import neurogym as ngym
from neurogym.wrappers import ScheduleEnvs
from neurogym.utils import scheduler
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from neurogym.wrappers.block import MultiEnvs
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import matplotlib
from matplotlib.colors import ListedColormap
import copy
import torch.nn.functional as F


/Users/ismaelrobles-razzaq/Desktop/research/.venv/lib/python3.11/site-packages/gym/envs/registration.py:396: UserWarning: WARN: The `registry.all` method is deprecated. Please use `registry.values` instead.
  logger.warn(


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Set up the environment
envid = 'yang19.dm1-v0'

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [4]:

config = {
    'dt': 100,
    'hidden_size': 20,
    'lr': 1e-3,
    'batch_size': 64,
    'seq_len': 10,
}

Vanilla dm1 RNN first

In [5]:
kwargs = {}
kwargs['timing'] = {
                'fixation': 300,
                'stimulus': 500,
                'decision': 200} # fix timing for discrete trial intervals

task_env = ngym.make('yang19.dm1-v0', **kwargs)
dataset = ngym.Dataset(task_env, batch_size=64, seq_len=10)
env = dataset.env

ob_size = env.observation_space.shape[0]
act_size = env.action_space.n

In [6]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, hidden = self.rnn(x)
        x = self.linear(out)
        return x, out

In [7]:
net = Net(input_size=env.observation_space.shape[0],
          hidden_size=config['hidden_size'],
          output_size=act_size)
net = net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=config['lr'])

In [68]:
# Training loop
net.train()
num_epochs = 10000
best_loss = float('inf')
#patience = 200
#no_improve_count = 0
correct = 0
total = 0
for epoch in range(num_epochs):
    net.train()
    total_loss = 0

    inputs, labels = dataset()
    inputs = torch.from_numpy(inputs).float().to(device)
    labels = torch.from_numpy(labels).long().to(device)

    optimizer.zero_grad()
    outputs, _ = net(inputs)
    loss = criterion(outputs.view(-1, act_size), labels.view(-1))
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    _, predicted = torch.max(outputs.data, -1)
    total += labels.size(1)
    for i in range(labels.size(1)):
        if torch.equal(predicted[:, i], labels[:, i]):
            correct += 1

    avg_loss = total_loss
    accuracy = 100 * correct / total

    if epoch % 500 == 499:
        print(f'Epoch {epoch + 1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')
        correct = 0
        total = 0

    if avg_loss < best_loss:
        best_loss = avg_loss
        #no_improve_count = 0
        torch.save(net.state_dict(), 'net_accuracy.pth')
    '''else:
        no_improve_count += 1

    if no_improve_count >= patience:
        print(f'Early stopping at epoch {epoch}')
        break'''

print('Finished Training')



Epoch 500, Loss: 0.0620, Accuracy: 89.84%
Epoch 1000, Loss: 0.0604, Accuracy: 90.18%
Epoch 1500, Loss: 0.0342, Accuracy: 90.03%
Epoch 2000, Loss: 0.0668, Accuracy: 90.17%
Epoch 2500, Loss: 0.0390, Accuracy: 90.25%
Epoch 3000, Loss: 0.0239, Accuracy: 90.00%
Epoch 3500, Loss: 0.0253, Accuracy: 90.17%
Epoch 4000, Loss: 0.0719, Accuracy: 90.40%
Epoch 4500, Loss: 0.0930, Accuracy: 90.21%
Epoch 5000, Loss: 0.0356, Accuracy: 90.60%
Epoch 5500, Loss: 0.0349, Accuracy: 90.05%
Epoch 6000, Loss: 0.0438, Accuracy: 90.15%
Epoch 6500, Loss: 0.0557, Accuracy: 90.46%
Epoch 7000, Loss: 0.0221, Accuracy: 90.21%
Epoch 7500, Loss: 0.0303, Accuracy: 90.12%
Epoch 8000, Loss: 0.0529, Accuracy: 90.43%
Epoch 8500, Loss: 0.0397, Accuracy: 90.55%
Epoch 9000, Loss: 0.0471, Accuracy: 90.33%
Epoch 9500, Loss: 0.0366, Accuracy: 90.30%
Epoch 10000, Loss: 0.0340, Accuracy: 90.10%
Finished Training


Get activities
Use batch size 1 for simplicity?
matrix should have input, label, model output, model activity for each unit at each timestep

In [16]:
inputs, labels = dataset()
print(labels.shape)
inputs.shape


(10, 1)


(10, 1, 33)

In [8]:
task_env = ngym.make('yang19.dm1-v0', **kwargs)
dataset = ngym.Dataset(task_env, batch_size=1, seq_len=10)
env = dataset.env

ob_size = env.observation_space.shape[0]
act_size = env.action_space.n

In [9]:
inputs, labels = dataset()
labels.shape

(10, 1)

In [10]:
def generate_matrix(model, dataset, num_trials=10000, timesteps=10):

    inputs, labels = dataset()

    input_size = inputs.shape[2]
    label_size = labels.shape[1]
    output_size = 1
    hidden_size = config['hidden_size']  
    timesteps = labels.shape[0]
    print(f"inputs_size: {input_size}, label_size: {label_size}, output_size: {output_size}, hidden_size: {hidden_size}, timesteps = {timesteps}")

    matrix_size = (num_trials, timesteps, input_size + label_size + output_size + hidden_size)
    output_matrix = np.zeros(matrix_size)
    model.eval()

    for trial in range(num_trials):

        inputs, labels = dataset()  

        inputs = torch.tensor(inputs, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.float32)

        with torch.no_grad():
            hidden_states, _ = model.rnn(inputs)
            output, _ = model(inputs)
            _, predicted = torch.max(output.data, -1) 


            for t in range(timesteps):
                output_matrix[trial, t, :input_size] = inputs[t, 0, :].numpy()
                output_matrix[trial, t, input_size:input_size+label_size] = labels[t].numpy()
                output_matrix[trial, t, input_size+label_size:input_size+label_size+output_size] = predicted[t, 0].numpy()
                output_matrix[trial, t, -hidden_size:] = hidden_states[0, 0, :].numpy()

    return output_matrix


In [12]:
# load trained model
net = Net(input_size=env.observation_space.shape[0],
          hidden_size=config['hidden_size'],
          output_size=act_size)

net.load_state_dict(torch.load('small_net.pth'))

net.eval()

/var/folders/9c/4b964bn507zc652npm2l2ch80000gn/T/ipykernel_21568/219679984.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('small_net.pth'

Net(
  (rnn): RNN(33, 20)
  (linear): Linear(in_features=20, out_features=17, bias=True)
)

In [33]:
# check correct output shapes
inputs = torch.tensor(inputs, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)

with torch.no_grad():
    hidden_states, _ = net.rnn(inputs)
    output, _ = net(inputs)
    _, predicted = torch.max(output.data, -1) 
predicted.shape


/var/folders/9c/4b964bn507zc652npm2l2ch80000gn/T/ipykernel_16631/3984640189.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32)
/var/folders/9c/4b964bn507zc652npm2l2ch80000gn/T/ipykernel_16631/3984640189.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float32)


torch.Size([10, 1, 17])

In [13]:


result_matrix = generate_matrix(net, dataset)
print(result_matrix.shape)  # Should be (10000, 10, 55)

inputs_size: 33, label_size: 1, output_size: 1, hidden_size: 20, timesteps = 10
(10000, 10, 55)


In [14]:
np.save('small_rnn', result_matrix)

Code for large 20 task vanilla rnn -- 100 length sequence should be good, no need for timestep lineup

In [28]:
config = {
    'dt': 100,
    'hidden_size': 256,
    'lr': 1e-3,
    'batch_size': 16,
    'seq_len': 100,
}

In [37]:
task_names = ['yang19.go-v0', 'yang19.rtgo-v0', 'yang19.dlygo-v0', 'yang19.anti-v0', 'yang19.rtanti-v0', 'yang19.dlyanti-v0', 'yang19.dm1-v0', 'yang19.dm2-v0', 'yang19.ctxdm1-v0', 'yang19.ctxdm2-v0', 'yang19.multidm-v0', 'yang19.dlydm1-v0', 'yang19.dlydm2-v0', 'yang19.ctxdlydm1-v0', 'yang19.ctxdlydm2-v0', 'yang19.multidlydm-v0', 'yang19.dms-v0', 'yang19.dnms-v0', 'yang19.dmc-v0', 'yang19.dnmc-v0']
task_list = []
for name in task_names:
    task_list.append(ngym.make(name))


# Create a schedule for switching between tasks
schedule = scheduler.RandomSchedule(n=len(task_list))

# Combine the tasks
combined_env = ScheduleEnvs(task_list, schedule=schedule, env_input=True)

# Create the dataset
dataset = ngym.Dataset(combined_env, batch_size=16, seq_len=100)
env = dataset.env

ob_size = env.observation_space.shape[0]
act_size = env.action_space.n

In [29]:
# Initialize network
multitask_net = Net(input_size=env.observation_space.shape[0],
          hidden_size=config['hidden_size'],
          output_size=act_size)
multitask_net = multitask_net.to(device)

# Set up loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(multitask_net.parameters(), lr=config['lr'])

In [54]:
# Training loop
multitask_net.train()
print('Training tasks:', [name for name in task_names])

num_epochs = 4000
best_loss = float('inf')
#patience = 200
#no_improve_count = 0

for epoch in range(num_epochs):
    multitask_net.train()
    total_loss = 0
    correct = 0
    total = 0

    inputs, labels = dataset()
    inputs = torch.from_numpy(inputs).float().to(device)
    labels = torch.from_numpy(labels).long().to(device)

    optimizer.zero_grad()
    outputs, _ = multitask_net(inputs)
    loss = criterion(outputs.view(-1, act_size), labels.view(-1))
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    _, predicted = torch.max(outputs.data, -1)
    total += labels.numel()
    correct += (predicted == labels).sum().item()

    avg_loss = total_loss
    accuracy = 100 * correct / total

    if epoch % 200 == 199:
        print(f'Epoch {epoch + 1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

    if avg_loss < best_loss:
        best_loss = avg_loss
        #no_improve_count = 0
        torch.save(multitask_net.state_dict(), 'multitask_net.pth')
    '''else:
        no_improve_count += 1

    if no_improve_count >= patience:
        print(f'Early stopping at epoch {epoch}')
        break'''

print('Finished Training')

Training tasks: ['yang19.go-v0', 'yang19.rtgo-v0', 'yang19.dlygo-v0', 'yang19.anti-v0', 'yang19.rtanti-v0', 'yang19.dlyanti-v0', 'yang19.dm1-v0', 'yang19.dm2-v0', 'yang19.ctxdm1-v0', 'yang19.ctxdm2-v0', 'yang19.multidm-v0', 'yang19.dlydm1-v0', 'yang19.dlydm2-v0', 'yang19.ctxdlydm1-v0', 'yang19.ctxdlydm2-v0', 'yang19.multidlydm-v0', 'yang19.dms-v0', 'yang19.dnms-v0', 'yang19.dmc-v0', 'yang19.dnmc-v0']
Epoch 200, Loss: 0.0460, Accuracy: 98.94%


KeyboardInterrupt: 

In [45]:
def calculate_accuracy(net, task_list, num_trials=10):
    net.eval()
    env = dataset.env
    env.reset()

    for i, task in enumerate(task_list):
        env.set_i(i)
        task_name = task_names[i]
        print(f"\nAnalyzing task: {task_name}")

        total_correct = 0
        total_decisions = 0
        non_fixation_correct = 0
        non_fixation_decisions = 0

        for _ in range(num_trials):
            with torch.no_grad():
                inputs, labels = dataset()
                inputs = torch.from_numpy(inputs).float().to(device)
                labels = torch.from_numpy(labels).long().to(device)

                outputs, _ = net(inputs)

                # Convert to numpy and process the entire batch
                inputs = inputs.cpu().numpy()
                labels = labels.cpu().numpy()
                outputs = outputs.cpu().numpy()

                # Calculate total accuracy
                predictions = np.argmax(outputs, axis=2)
                total_correct += np.sum(predictions == labels)
                total_decisions += labels.size

                # Find non-fixation periods (when fixation input is 0)
                non_fixation = inputs[:, :, 0] == 0

                # Calculate non-fixation accuracy
                non_fixation_predictions = predictions[non_fixation]
                non_fixation_labels = labels[non_fixation]
                non_fixation_correct += np.sum(non_fixation_predictions == non_fixation_labels)
                non_fixation_decisions += non_fixation_labels.size

        total_accuracy = total_correct / total_decisions if total_decisions > 0 else 0
        non_fixation_accuracy = non_fixation_correct / non_fixation_decisions if non_fixation_decisions > 0 else 0

        print(f"Total accuracy ({total_decisions} decisions): {total_accuracy:.4f}")
        print(f"Non-fixation accuracy ({non_fixation_decisions} decisions): {non_fixation_accuracy:.4f}")

In [55]:
calculate_accuracy(multitask_net, task_list, num_trials=10)


Analyzing task: yang19.go-v0
Total accuracy (16000 decisions): 0.9840
Non-fixation accuracy (3237 decisions): 0.9209

Analyzing task: yang19.rtgo-v0
Total accuracy (16000 decisions): 0.9846
Non-fixation accuracy (3303 decisions): 0.9258

Analyzing task: yang19.dlygo-v0
Total accuracy (16000 decisions): 0.9819
Non-fixation accuracy (3205 decisions): 0.9095

Analyzing task: yang19.anti-v0
Total accuracy (16000 decisions): 0.9828
Non-fixation accuracy (3253 decisions): 0.9152

Analyzing task: yang19.rtanti-v0
Total accuracy (16000 decisions): 0.9854
Non-fixation accuracy (3255 decisions): 0.9284

Analyzing task: yang19.dlyanti-v0
Total accuracy (16000 decisions): 0.9852
Non-fixation accuracy (3211 decisions): 0.9265

Analyzing task: yang19.dm1-v0
Total accuracy (16000 decisions): 0.9841
Non-fixation accuracy (3319 decisions): 0.9235

Analyzing task: yang19.dm2-v0
Total accuracy (16000 decisions): 0.9819
Non-fixation accuracy (3135 decisions): 0.9075

Analyzing task: yang19.ctxdm1-v0
Tota

In [20]:
# change batch size to 1 for simplicity
task_names = ['yang19.go-v0', 'yang19.rtgo-v0', 'yang19.dlygo-v0', 'yang19.anti-v0', 'yang19.rtanti-v0', 'yang19.dlyanti-v0', 'yang19.dm1-v0', 'yang19.dm2-v0', 'yang19.ctxdm1-v0', 'yang19.ctxdm2-v0', 'yang19.multidm-v0', 'yang19.dlydm1-v0', 'yang19.dlydm2-v0', 'yang19.ctxdlydm1-v0', 'yang19.ctxdlydm2-v0', 'yang19.multidlydm-v0', 'yang19.dms-v0', 'yang19.dnms-v0', 'yang19.dmc-v0', 'yang19.dnmc-v0']
task_list = []
for name in task_names:
    task_list.append(ngym.make(name))


# Create a schedule for switching between tasks
schedule = scheduler.RandomSchedule(n=len(task_list))

# Combine the tasks
combined_env = ScheduleEnvs(task_list, schedule=schedule, env_input=True)

# Create the dataset
dataset = ngym.Dataset(combined_env, batch_size=1, seq_len=100)
env = dataset.env

ob_size = env.observation_space.shape[0]
act_size = env.action_space.n

In [32]:
multitask = torch.load('multitask_net.pth')

/var/folders/9c/4b964bn507zc652npm2l2ch80000gn/T/ipykernel_21568/3263536786.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  multitask = torch.load('multitask_net.pth')


In [33]:
multitask_matrix = generate_matrix(multitask_net, dataset)


inputs_size: 53, label_size: 1, output_size: 1, hidden_size: 256, timesteps = 100


In [34]:
np.savez_compressed('multitask_rnn', multitask_matrix)

In [ ]:


# Create a sample array
original_array = np.random.rand(1000, 1000)

# Save using savez_compressed
np.savez_compressed('compressed_file.npz', data=original_array)

# Load the compressed file
loaded_file = np.load('compressed_file.npz')
loaded_array = loaded_file['data']

# Check if shapes are the same
print("Original shape:", original_array.shape)
print("Loaded shape:", loaded_array.shape)

# Check if contents are the same
print("Arrays are identical:", np.array_equal(original_array, loaded_array))